In [ ]:
!apt updateapt update
!pip install mdbtools-dev
!apt-get install mdbtools
!pip install pandas pyodbc
import warnings
warnings.filterwarnings('ignore')


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import sys
sys.path.append('/content/drive/MyDrive/Dev/py2/')
import numpy as np

import pandas as pd
from google.colab import files
import os
import subprocess
from io import StringIO  # StringIOを正しくインポート
import model.utility.k_analysis_util as k_a_util
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt

In [7]:

# 使用例
table_name = 'PredictorVariables'  # テーブル名を指定

file_list = [f'kjdb_prediction_cache_{year}.accdb' for year in range(2000, 2025)]
years = range(2000, 2025)

k_a_util.prepare_pd_on_notebook()
print(len(file_list))

dict_statistics={}
for y in years:
  filepath = f'/content/drive/MyDrive/Dev/Kjvan/kjdb_prediction_cache_{y}.accdb'
  df_summary = k_a_util.read_table(filepath, table_name)
  columns_to_drop = [ 'key_program_id','key_horse_id', 'obj_rank',  'desc_horse_no', 'upd']
  df = df_summary.drop(columns=columns_to_drop)
  dict_statistics[y] = df

25


In [ ]:
print(len(dict_statistics[2000].columns))

In [ ]:
# ヒストグラムの作成 (グラフの行数、列数、解像度を適宜変更)
k_a_util.plot_histograms(dict_statistics, rows=8, cols=8, dpi=72)

In [20]:

def pivot_statistics_by_column(dict_statistics: dict, columns: list = None) -> dict:
    """
    統計データのDataFrameを列単位で転置し、新しいDataFrameを作成します。

    Args:
        dict_statistics: 年をキーとし、統計データのDataFrameを値とする辞書。
        columns: 転置する列名を指定します。Noneの場合、すべての列を転置します。

    Returns:
        転置された統計データのDataFrameを要素とする辞書。
    """

    def pivot_for_column(col: str) -> pd.DataFrame:
        """特定の列を転置する関数"""
        try:
            data = [df[col].values for df in dict_statistics.values()]
            df_pivot = pd.DataFrame(data, index=dict_statistics.keys()).T
            df_pivot.columns.name = 'Year'
            return df_pivot
        except KeyError:
            print(f"列 '{col}' は存在しません。")
            return pd.DataFrame()

    if columns is None:
        columns = dict_statistics[next(iter(dict_statistics))].columns

    return {col: pivot_for_column(col) for col in columns}

In [ ]:
df_summary = pivot_statistics_by_column(dict_statistics)

k_a_util.plot_histograms(df_summary, rows=8, cols=4, dpi=72)